<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/BoneAgePredictor_limit128_Grad_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kittimaxz/Project_BoneAge.git
!cd Project_BoneAge

Cloning into 'Project_BoneAge'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 463 (delta 4), reused 0 (delta 0), pack-reused 446
Receiving objects: 100% (463/463), 92.59 MiB | 44.16 MiB/s, done.
Resolving deltas: 100% (228/228), done.
Updating files: 100% (44/44), done.


In [2]:
import Project_BoneAge

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
import pandas as pd
from PIL import Image

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def dataset_path(*rel_path):
    return os.path.join('/content/drive/My Drive/Project_Boneage', *rel_path);

In [6]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [7]:
#from google.colab import files
#files.upload()
#import helper

In [8]:
class BoneAgeTrainingDataset(Dataset):
    def __init__(self,csv_path,img_folder):
        self.csv = pd.read_csv(dataset_path(csv_path));
        self.img_folder = img_folder;        

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, idx):
        male = np.array([1]) if (self.csv['male'][idx] == 'TRUE') else np.array([0])
        return transform(Image.open(dataset_path(self.img_folder,str(self.csv['id'][idx])+'.png')).resize((256,256))).double(),\
               torch.from_numpy(np.array(self.csv['boneage'][idx])).double(),\
               torch.from_numpy(male).double()

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target, male) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, male)
        loss = F.l1_loss(output.view(-1), target)
        loss.backward()
        optimizer.step()
        #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                #epoch, batch_idx * len(data), len(train_loader.dataset),
                #100. * batch_idx / len(train_loader), loss.item()))

In [10]:
class BoneAgePredictor(nn.Module):
    def __init__(self):
        super(BoneAgePredictor, self).__init__()
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, 3)
        nn.init.kaiming_normal_(self.conv1.weight)
        self.batch1 = nn.BatchNorm2d(16)
        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, 3)
        nn.init.kaiming_normal_(self.conv2.weight)
        self.batch2 = nn.BatchNorm2d(32)
        # Layer 3
        self.conv3 = nn.Conv2d(32, 64, 3)
        nn.init.kaiming_normal_(self.conv3.weight)
        self.batch3 = nn.BatchNorm2d(64)
        # Layer 4
        self.conv4 = nn.Conv2d(64, 128, 3)
        nn.init.kaiming_normal_(self.conv4.weight)
        self.batch4 = nn.BatchNorm2d(128)
        # Layer 5
        self.conv5 = nn.Conv2d(128, 128, 3)
        nn.init.kaiming_normal_(self.conv5.weight)
        self.batch5 = nn.BatchNorm2d(128)
        # Fully connected
        self.fc1 = nn.Linear(4609, 68)
        self.fc2 = nn.Linear(69, 1)
        #self.fc2 = nn.Linear(64, 1)
    
        

    def forward(self, x, m):
        # Layer 1
        x = F.relu(self.batch1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        # Layer 2
        x = F.relu(self.batch2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        # Layer 3
        x = F.relu(self.batch3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        # Layer 4
        x = F.relu(self.batch4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        # Layer 5
        x = F.relu(self.batch5(self.conv5(x)))
        x = F.max_pool2d(x, 2)
        
        # Pooling
        x = x.view(-1,4608)
        x = torch.cat((x,m), axis = 1)
        x = self.fc1(x)
        x = torch.cat((x,m), axis = 1)
        x = self.fc2(x)
        #x = torch.cat((x,m), axis = 1)
        #x = self.fc3(x)
        return x

In [11]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
trainig_data_loader = torch.utils.data.DataLoader(
    BoneAgeTrainingDataset('boneage-training-dataset_261.csv', 'boneage-training-dataset-261'),
    batch_size=32, shuffle=True, **kwargs)

In [12]:
model = BoneAgePredictor().double().to(device)
print(model)
optimizer = optim.Adam(model.parameters())
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=2, min_lr=0.0001, verbose=True)

BoneAgePredictor(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (batch1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (batch2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4609, out_features=68, bias=True)
  (fc2): Linear(in_features=69, out_features=1, bias=True)
)



#Grad-CAM


In [13]:
checkpoint = torch.load('/content/drive/My Drive/Project_Boneage/model-weight/model1_280323.pth')
checkpoint

{'epoch': 20,
 'train_loss': [0.641347200688918,
  0.5159797099663805,
  0.44174221461611823,
  0.42023026687186715,
  0.40796042196927185,
  0.34297480570943706,
  0.31950184376745944,
  0.3202581626852461,
  0.3093078207443282,
  0.28407029593441036,
  0.3087003803918411,
  0.2870299163303687,
  0.31500758143564983,
  0.1941060950087309,
  0.18140323980230594,
  0.17504718413999854,
  0.1681633670500109,
  0.16379147989612503,
  0.16719018329491586,
  0.15290479154629813],
 'val_loss': [0.582115090436791,
  0.4369857893799711,
  0.40350581592145957,
  0.39922859153810525,
  0.40511192794872,
  0.3361188779488728,
  0.3226191955426586,
  0.34152689345399145,
  0.34445401077613896,
  0.32157229349140837,
  0.34886308420216433,
  0.3507450752542824,
  0.36148263410924747,
  0.2905868962195608,
  0.29305108357492277,
  0.2905931517654777,
  0.28949777761353584,
  0.2922524561726568,
  0.29645288200066355,
  0.29074059238582306],
 'state_dict': OrderedDict([('conv1.weight',
              

In [14]:
import torch.nn as nn
import torch.optim as optim

# Load your pre-trained model
model = model
model.load_state_dict(checkpoint['state_dict'])

# Set the model to evaluation mode
model.eval()


BoneAgePredictor(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (batch1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (batch2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4609, out_features=68, bias=True)
  (fc2): Linear(in_features=69, out_features=1, bias=True)
)

In [36]:
layer = model.conv5  # replace with your model's last convolutional layer
layer

Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))

In [37]:
input = transform(Image.open(dataset_path('boneage_training_dataset/1377.png')).resize((256,256))).double()  # replace with your input
input

tensor([[[0.9451, 0.9412, 0.7765,  ..., 0.2745, 0.2784, 0.2824],
         [0.9451, 0.9451, 0.7843,  ..., 0.2706, 0.2706, 0.2784],
         [0.9451, 0.9451, 0.7882,  ..., 0.2667, 0.2706, 0.2745],
         ...,
         [0.8784, 0.8824, 0.8745,  ..., 0.1765, 0.1804, 0.1804],
         [0.8824, 0.8824, 0.8863,  ..., 0.1843, 0.1843, 0.1843],
         [0.8863, 0.8745, 0.8824,  ..., 0.1922, 0.1882, 0.1882]]],
       dtype=torch.float64)

In [38]:
np.shape(input)

torch.Size([1, 256, 256])

In [39]:
#input = input[None,:,:,:]
np.shape(input)

torch.Size([1, 256, 256])

In [40]:
m = torch.from_numpy(np.array([1.0])).double() # given that it's a picture of "male"
m

tensor([1.], dtype=torch.float64)

In [41]:
input = input.resize(1,1,256,256)
input

/usr/local/lib/python3.9/dist-packages/torch/_tensor.py:761: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([[[[0.9451, 0.9412, 0.7765,  ..., 0.2745, 0.2784, 0.2824],
          [0.9451, 0.9451, 0.7843,  ..., 0.2706, 0.2706, 0.2784],
          [0.9451, 0.9451, 0.7882,  ..., 0.2667, 0.2706, 0.2745],
          ...,
          [0.8784, 0.8824, 0.8745,  ..., 0.1765, 0.1804, 0.1804],
          [0.8824, 0.8824, 0.8863,  ..., 0.1843, 0.1843, 0.1843],
          [0.8863, 0.8745, 0.8824,  ..., 0.1922, 0.1882, 0.1882]]]],
       dtype=torch.float64)

In [42]:
m = m.resize(1,1)
m

tensor([[1.]], dtype=torch.float64)

In [43]:
output = model(input, m)
output

tensor([[1.4652]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [23]:
features = []
def get_features_hook(model, input, output):
    features.append(output)

hook_handle = model.conv5.register_forward_hook(get_features_hook)
output = model(input,m)
hook_handle.remove()

feature_maps = features[0]
gradients = model.conv5.weight.grad

In [35]:
weights = torch.mean(gradients, axis=(2, 3))
weights

TypeError: ignored

In [ ]:
cam = torch.zeros(feature_maps.shape[2:], dtype=torch.float32)
cam

In [ ]:
for i, w in enumerate(weights[0]):
    cam += w * feature_maps[0, i, :, :]
cam = torch.nn.functional.relu(cam)
cam

In [ ]:
cam = cam.detach().numpy()
cam

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(input.squeeze().detach().numpy())
ax[0].axis('off')
ax[0].set_title('Input Image')
ax[1].imshow(cam)#, cmap='jet')
ax[1].axis('off')
ax[1].set_title('GradCAM Heatmap')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(input.squeeze().detach().numpy())
#ax[0].axis('off')
ax[0].set_title('Input Image')
ax[1].imshow(cam, cmap='jet')
#ax[1].axis('off')
ax[1].set_title('GradCAM Heatmap')
plt.show()

In [ ]:
import skimage.transform

In [ ]:
input.shape

In [ ]:
cam.shape

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))

ax[0].imshow(input.squeeze().detach().numpy())
#ax[0].set_title("Video: " + sample_video + "Actual: " + y )

ax[1].imshow(input.squeeze().detach().numpy())
ax[1].imshow(skimage.transform.resize(cam, (input.shape[2],input.shape[3] )), alpha=0.5, cmap='jet')

#plt.imshow(cam.detach().numpy(), alpha=0.5,cmap='jet')
#y_pred = str(y_pred.cpu().data.numpy())
#ax[1].set_title(y_pred)
#fig.tight_layout()

In [ ]:
plt.imshow(input.squeeze().detach().numpy())
plt.imshow(skimage.transform.resize(cam, (input.shape[2],input.shape[3] )), alpha=0.5, cmap='jet')
plt.colorbar()
#plt.imshow(cam.detach().numpy(), alpha=0.5,cmap='jet')
#y_pred = str(y_pred.cpu().data.numpy())
#ax[1].set_title(y_pred)
#fig.tight_layout()


#docx

In [ ]:
def hook_fn(module, input, output):
    # Save the output of the layer to a global variable
    global feature_map
    feature_map = output.detach()


In [ ]:
# Assuming your CNN model is called "model"
last_conv_layer = model.conv5
hook_handle = last_conv_layer.register_forward_hook(hook_fn)

In [ ]:
gradients = model.conv5.weight.grad

In [ ]:
weights = torch.mean(gradients, axis=(2, 3))
cam = torch.zeros(feature_maps.shape[2:], dtype=torch.float32)

In [ ]:
prediction = model(input,m)  # x is the input X-ray image
prediction.backward()  # Backpropagate to compute gradients

# Calculate the mean gradient over the feature map
#grads = feature_map.grad.mean(dim=(2, 3), keepdim=True)

# Multiply the feature map by the gradients and take the mean along the channel dimension
cam = (feature_map * cam).sum(dim=1, keepdim=True)


In [ ]:
import matplotlib.pyplot as plt
from torchvision.transforms.functional import resize

cam = cam.detach().squeeze()
cam = (cam - cam.min()) / (cam.max() - cam.min())
#cam = resize(cam, size=input.size()[-2:])
plt.imshow(cam, cmap='jet')


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,10))

ax[0].imshow(input.squeeze().detach().numpy())
#ax[0].set_title("Video: " + sample_video + "Actual: " + y )

ax[1].imshow(input.squeeze().detach().numpy())
ax[1].imshow((cam, (input.shape[0],input.shape[1] )), alpha=0.1, cmap='jet')

#y_pred = str(y_pred.cpu().data.numpy())
#ax[1].set_title(y_pred)
#fig.tight_layout()